<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/N_prediciont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

**N 예측**

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers>=4.8.1, but you have transformers 3.0.2 which is incompatible.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-adiy0ohl
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-adiy0ohl
  Using cached transformers-4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 산업분류자동화		 df_E_inc.csv	   F_inc_model_2.pt   N_inc_model.pt
'Colab Notebooks'	'# df_G_inc.csv'   G_inc_model.pt     P_inc_model.pt
'답안 작성용 파일.csv'	 df_J_inc.csv	   I_ori_model.pt     Q_inc_model.pt
 df_A_inc.csv		 df_K_inc.csv	   J_inc_model.pt     S_ori_model.pt
 df_B_inc.csv		 df_N_inc.csv	  'My Drive'	      데이터.zip


In [ ]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
replace /content/dataset/2. ╕Ё╡и┐ы░│╣▀┐ы└┌╖с.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset/2. ╕Ё╡и┐ы░│╣▀┐ы└┌╖с.csv  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
replace /content/dataset/1. 실습용자료.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset/1. 실습용자료.txt  


In [ ]:
import pandas as pd
ts=pd.read_csv('/content/gdrive/MyDrive/답안 작성용 파일.csv' ,encoding = "euc-kr")

In [ ]:
test=ts[ts['digit_1']=='N']
test=test.reset_index(drop=True)

# 제출용 파일 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
model = torch.load('/content/gdrive/MyDrive/N_inc_model.pt')

model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)  
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        return np.argmax(logits)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 제출하기

In [ ]:
test['text']=test['text_obj'].map(str)+" "+test['text_mthd'].map(str)+" "+test['text_deal'].map(str)

i=[]
for sentence in test['text']:
    a=sentence.replace('nan', '')
    i.append(a)
test['text']=pd.DataFrame(i)

test=test.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

test.head(3)

,AI_id,digit_1,digit_2,digit_3,text
0,id_000175,N,NaN,NaN,사업장에서 관련산업사용자대상으로 건설용기계 임대업
1,id_000238,N,NaN,NaN,정수기 고객대상 정수기렌탈. 필터교환
2,id_000329,N,NaN,NaN,"기타 사무지원 고객요구로 청구서작성,일일사무행정 서비스"


In [ ]:
result=[]
for sentence in test['text']:
  temp=predict(sentence)
  result.append(temp)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
a=pd.DataFrame(result, columns=['digit_3'])
a.loc[(a['digit_3'] == 0), 'digit_3'] = 741
a.loc[(a['digit_3'] == 1), 'digit_3'] = 742 
a.loc[(a['digit_3'] == 2), 'digit_3'] = 743
a.loc[(a['digit_3'] == 3), 'digit_3'] = 751  
a.loc[(a['digit_3'] == 4), 'digit_3'] = 752  
a.loc[(a['digit_3'] == 5), 'digit_3'] = 753  
a.loc[(a['digit_3'] == 6), 'digit_3'] = 759 
a.loc[(a['digit_3'] == 7), 'digit_3'] = 761
a.loc[(a['digit_3'] == 8), 'digit_3'] = 762
a.loc[(a['digit_3'] == 9), 'digit_3'] = 763
a.loc[(a['digit_3'] == 10), 'digit_3'] = 764

In [ ]:
test['digit_3']=a

In [ ]:
test.to_csv('result_N_inc.csv', index=False, encoding='utf-8-sig')

**Q 예측**

In [ ]:
import pandas as pd
ts=pd.read_csv('/content/gdrive/MyDrive/답안 작성용 파일.csv' ,encoding = "euc-kr")
test=ts[ts['digit_1']=='Q']
test=test.reset_index(drop=True)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
model = torch.load('/content/gdrive/MyDrive/Q_inc_model.pt')

model.eval()

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)  
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        return np.argmax(logits)

In [ ]:
test['text']=test['text_obj'].map(str)+" "+test['text_mthd'].map(str)+" "+test['text_deal'].map(str)

i=[]
for sentence in test['text']:
    a=sentence.replace('nan', '')
    i.append(a)
test['text']=pd.DataFrame(i)

test=test.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

test.head(3)

In [ ]:
result=[]
for sentence in test['text']:
  temp=predict(sentence)
  result.append(temp)

In [ ]:
a=pd.DataFrame(result, columns=['digit_3'])
a.loc[(a['digit_3'] == 0), 'digit_3'] = 861
a.loc[(a['digit_3'] == 1), 'digit_3'] = 862  
a.loc[(a['digit_3'] == 2), 'digit_3'] = 863
a.loc[(a['digit_3'] == 3), 'digit_3'] = 869  
a.loc[(a['digit_3'] == 4), 'digit_3'] = 871  
a.loc[(a['digit_3'] == 5), 'digit_3'] = 872

In [ ]:
test['digit_3']=a

In [ ]:
test.to_csv('result_Q.csv', index=False, encoding='utf-8-sig')